In [ ]:
class Movalyst:
    def __init__(self):
        return self
    # get all urls for movie list from year x to year y
    def get_movie_url(x=1980,y=2018):
        import requests 
        import datetime
        from bs4 import BeautifulSoup
    
        if x > y :
            raise ValueError('Start time should be earlier.')
        if x < 1980 or y> int(str(datetime.datetime.now())[:4]):
            raise ValueError('Please input time during 1980 till now.')
    
        url = 'https://www.boxofficemojo.com/yearly/'
        url_list = []
    
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')
    
        for result in results:
            if 'yr=' in result.get('href') and int(result.get_text())>=x and int(result.get_text())<=y:
                url_list.append(url+result.get('href'))
        return url_list

    def get_movie(url = None, year = None):
        import requests 
        from bs4 import BeautifulSoup
        if url == None:
            url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')        
    
        url_list=[]
        for result in results:
            if result.get('href') != None:
                if 'id='in result.get('href') and '#' not in result.get_text():
                    url_list.append('https://www.boxofficemojo.com'+result.get('href'))
        return url_list
    
    def get_movie_name(url = None, year = None):
        import requests 
        from bs4 import BeautifulSoup
        if url == None:
        url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')        
    
        moviename_list=[]
        for result in results:
            if result.get('href') != None:
                if 'id='in result.get('href') and '#' not in result.get_text():
                    moviename_list.append(result.get_text())
        return moviename_list
    
    def find_all(x,y):
        import requests 
        from bs4 import BeautifulSoup
        list_url = get_movie_url(x,y)
        dict_movie = {}
        for url in list_url:
            list_0 = list_movie.copy()
            list_1 = dict(map(lambda x,y:[x,y],get_movie_name(url),get_movie(url)))
            dict_movie = {**list_0, **list_1}
            return dict_movie
    
#     def data_movie(year):
        
#         import requests 
#         from bs4 import BeautifulSoup
#         url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
#         response = requests.get(url)
#         results = BeautifulSoup(response.content, 'lxml')
#         results = results.find_all('font',size="2")
#         return results
    
    # get a movie's daily gross
    def gross_movie(name,begin_year=1980,end_year=2018):      
        import pandas as pd
        import requests 
        from bs4 import BeautifulSoup
        try:
            movie_url = find_all(begin_year,end_year)[name]
            id_position = movie_url.find('id=')
            name_id = movie_url[id_position:]
            url = 'https://www.boxofficemojo.com/movies/?page=daily&view=chart&'+name_id
            response = requests.get(url)
            results = BeautifulSoup(response.content, 'lxml')
            tables = results.select('table')
            df_list = []
            for table in tables:
                df_list.append(pd.concat(pd.read_html(table.prettify())))
            movie_gross = df_list[7]
            movie_gross.columns = ['Day','Date','Rank','Gross','%daily-change','%weekly-change','Theaters','Average','Gross-to-Date','Days']
            movie_gross = movie_gross[(movie_gross['Date'] != 'Date') & (movie_gross['Date'].notnull())]
            movie_gross.set_index('Date',inplace=True)
            movie_gross[['Days']] = movie_gross[['Days']].astype(int)
            return movie_gross        
        except:
            ValueError(f'{name} is not in the TOP100  of year from to {begin_year} to {end_year}')
    
    # get a movie's basic information as a tabel and put the result into a csv
    def data_movie(name,begin_year=1980,end_year=2018):      
        import pandas as pd
        import requests 
        from bs4 import BeautifulSoup
        try:
            movie_url = find_all(begin_year,end_year)[name]
            response = requests.get(movie_url)
            results = BeautifulSoup(response.content, 'lxml')
            tables = results.select('table')
            df_list = []
            for table in tables:
                df_list.append(pd.concat(pd.read_html(table.prettify())))
            movie_info = {}
            movie_info['Name'] = name
            movie_info['Genre'] = df_list[5][0].iloc[2][8:]
            movie_info['Runtime'] = df_list[5][1].iloc[2][10:]
            movie_info['Production Budget'] = df_list[5][1].iloc[3][20:]
            movie_info['Release Date'] = df_list[5][1].iloc[1][15:]
            movie_info['Close Date'] = df_list[12][1].iloc[0] 
            movie_info['Inrelease Time'] = df_list[13][1].iloc[0] 
            movie_info['Widest Release'] = df_list[11][1].iloc[0] 
            movie_info['Domestic Gross'] = df_list[9][1].iloc[0]
            movie_info['Foreign Gross'] = df_list[9][1].iloc[1]
            movie_info['Worldwide Gross'] = df_list[9][1].iloc[3]
            df_list[14].set_index(0,inplace=True)
            try:
                movie_info['Director'] = df_list[14][1].loc['Director:']
            except:
                movie_info['Director'] = df_list[14][1].loc['Directors:']
            movie_info['Actors'] = df_list[14][1].loc['Actors:']
            try:
                movie_info['Producer'] = df_list[14][1].loc['Producer:']
            except:
                movie_info['Producer'] = df_list[14][1].loc['Producers:']
            movie_info['Distributor'] = df_list[5][0].iloc[1][14:]
            mv = pd.DataFrame(list(movie_info.items()))
            mv.columns = ['Items','basic info']
            mv.set_index('Items',inplace=True)
#             mv.to_csv(path_or_buf=f'./{name}Basic_info.csv')
            return mv
        except:
            ValueError(f'{name} is not in the TOP100 of year from to {begin_year} to {end_year}')
    
    
    #get a list of movies' information tables and input them into a csv
    def data_movies_table(begin_year=1980,end_year=2018,input_name=None):  
        if not input_name == None:
            name_list = input_name
        else:
            name_list = list(find_all(begin_year,end_year).keys())
        movie_table = data_movie(name_list[0],begin_year,end_year)
        for name in name_list[1:]:
            try:
            movie_info = data_movie(name,begin_year,end_year)
            movie_table[name]=movie_info[name]
            except:
                continue
#         mv.to_csv(path_or_buf=f'./{begin_year}-{end_year}some movies basic info.csv')
        return movie_table      

    def get_review(name, page_length = 2, num = 20):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://www.rottentomatoes.com/m/'+name+'/reviews/'
        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        results = results_page.find_all('div', class_ = 'the_review')
        page = 1
        reviews = list()
        next_page_results = list()

        while page < page_length:
            for item in results_page.find_all('a', class_ = 'btn btn-xs btn-primary-rt'):
                if item.find('span', class_ = "glyphicon glyphicon-chevron-right"):
                    url = 'https://www.rottentomatoes.com' + item.get('href')
                    response = requests.get(url)
                    results_page = BeautifulSoup(response.content, 'lxml')
                    next_page_results = results_page.find_all('div', class_ = 'the_review')
            results.extend(next_page_results)
            page+=1
        for result in results:
            reviews.append(result.get_text())

        return reviews[:num]

    def get_name_review(year, num = 30, page_length = 2, length = 20):
        movies = get_movie_name(None, year)
        for i in range(len(movies)):
            movies[i] = movies[i].lower().replace(' ', '_').replace(':', '').replace(',', '_').replace('.', '_').replace("'", '').replace('!', '_').replace('?', '').replace('-', '_').replace('(', '').replace(')', '').replace('_a_new_dragon_tattoo_story','').replace('the_equalizer_2','equalizer_2').replace('dr__seuss_the_grinch_2018','the_grinch').replace('disneys_','')
            movies[i] = movies[i].lower().replace('___', '_')
            movies[i] = movies[i].lower().replace('__', '_')
        review_list = list()
        name_list = get_movie_name(None, year)[:num]
        for movie in movies[:num]:
            review = get_review(movie, page_length, length)
            if not review or len(review)<5:
                try:
                    review = get_review(movie+f'_{year}', page_length, length)
                except:
                    print('Do not find the movie')
        
            review_list.append(' '.join(review))
        name_review = list(zip(name_list, review_list))
        return name_review



In [ ]:
# There are some details need to be adjusted
# get_movie_urls: the first item is wrong
# data_movies_table: something is wrong when handling with lots of items
